In [3]:
"""
Tokenization and sentiment analysis
"""
# conn.cursor().execute("""DROP TABLE linkusersentimenttosubreddit""")
sql_create_linkusersentimenttosubreddit_table = """ CREATE TABLE IF NOT EXISTS linkusersentimenttosubreddit (
                                        userid text,
                                        subredditid text,
                                        rating integer
                                    ); """
conn.cursor().execute(sql_create_linkusersentimenttosubreddit_table)

"""
The snippet below is ran onced - to insert data into a new table
"""

# sid = SentimentIntensityAnalyzer()
# for comment in reddit_comments:
#     array_of_sentences = nltk.sent_tokenize(comment[2])
#     comment_sentiment = 0
#     for sentence in array_of_sentences:
#         ss = sid.polarity_scores(sentence)
#         array_of_tokenized_values = []
#         comment_sentiment += ss['pos']  
#     conn.cursor().execute("INSERT INTO linkusersentimenttosubreddit VALUES (?, ?, ?)", (comment[1], comment[0], comment_sentiment))
#     conn.commit()
newtable = conn.cursor().execute("SELECT * from linkusersentimenttosubreddit").fetchall()
comments_df = pd.DataFrame(newtable, columns = ['UserID', 'SubredditID', 'Rating'])
R_df = comments_df.pivot(index = 'UserID', columns ='SubredditID', values = 'Rating').fillna(0)
R_df.head()

# # """
# # convert to a numpy array and normalize data
# # """
R = R_df.as_matrix()
print(R)
ratings_mean = np.mean(R)
ratings_mean.reshape(-1,1)
R_demeaned = R - ratings_mean.reshape(-1, 1)

[[ 0.     0.     0.     0.   ]
 [ 0.     0.     0.423  0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]]


In [2]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse.linalg import svds


def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

conn = create_connection("database.sqlite")
    
reddit_comments = conn.cursor().execute("SELECT subreddit_id, name, body  from May2015 LIMIT 5").fetchall()
reddit_comments[0]

/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


('t5_378oi', 't1_cqug90g', 'くそ\n読みたいが買ったら負けな気がする\n図書館に出ねーかな')

In [12]:
"""
Alright now to perform svd
"""
U, sigma, Vt = svds(R_demeaned, k = 2)
sigma = np.diag(sigma)

all_users_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + ratings_mean.reshape(-1,1)
preds_df = pd.DataFrame(all_users_predicted_ratings, columns = R_df.columns)

In [ ]:
"""
Alright lets make the recommendation engine
"""
def recommend_subreddits(predictions_df, userID, comments_df, original_ratings_df, num_recommendations=5):
    #Get and sort user's predictions
    user_row_number = userID - 1
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    #Get user's data and merge in the comments data
    user_Data = N
    